## Notions de base

### Vocabulaire

Conteneur 
: Un conteneur est une sorte de bac à sable (sandbox) qui isole un ensemble de processus du système en utilisant les namespaces du noyau linux et leur associe une carte réseau virtuelle. Un conteneur est l'instance d'une image. Généralement, un conteneur fournit un et un seul service.

Image
: Une image est système de fichier isolé qui contient tout ce qui est nécessaire pour l'exécution d'un conteneur. Elle contient aussi d'autres informations comme des metadonnées, des variables d'environnement ou la commande par défaut à exécuter. Souvent une image s'appuie sur une autre image à laquelle elle apporte des modificiations.

Docker Daemon
: Le docker daemon est un serveur qui gère les images, les conteneurs, les réseaux et les volumes de stockage. Il peut communiquer avec d'autres docker daemons et offre une API standard.

Docker Client
: Le docker client est un outils en ligne de commande pour envoyer des commandes à un ou plusieurs docker daemons locaux ou distants. 


> Par exemple, un docker client peut demander à un docker daemon d'exécuter une image ce qui produit un conteneur. Si l'image n'est pas disponible localement, elle est téléchargée par le docker daemon depuis un registry.

### Installation

Docker peut être installé directement sous linux, il partage alors le noyau de l'hôte. Il peut aussi être installé dans une machine virtuelle Linux (sous Linux, Windows et MacOs) par exemple via [Docker Desktop](https://docs.docker.com/get-docker/).

### Exécuter un conteneur

La commande `docker container run` permet d'executer la commande par défaut d'une image dans un espace isolé. Si l'image n'est pas disponible pour le docker daemon, elle est téléchargée à partir d'un registry (par défaut [Docker Hub](http://hub.docker.com)).

Comme premier exemple, exécutons l'image [hello-world](https://hub.docker.com/_/hello-world) qui illustre ce fonctionnement (lire le résultat) :

In [7]:
docker container run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



Une commande docker commence par le type d'objet qu'elle manipule (`container`, `image`, ...) suivi d'une sous-commande. Par exemple, il est possible de demander au docker daemon de précharger ou de mettre à jour une image avec la sous-commande `pull`.

Pour télécharger ou mettre à jour [Alpine Linux](https://www.alpinelinux.org/) qui est une distribution linux très légère :

In [3]:
docker image pull alpine

Using default tag: latest
latest: Pulling from library/alpine
Digest: sha256:69665d02cb32192e52e07644d76bc6f25abeb5410edc1c7a81a10ba3f0efb90a
Status: Image is up to date for alpine:latest
docker.io/library/alpine:latest


Le format du nom d'une image est `[registry_hostname[:port]][path]image_name[:tag]`. Si l'adresse du registry est omise c'est celle de dockerhub qui est utilisé (docker.io). Si l'image est "officielle" `path` peut-être omis et vaut `library`, sinon il s'agit généralement du compte de l'utilisateur qui fourni l'image. `tag` permet de différencier les versions d'une image, s'il est omis il vaut `latest`. 

Ainsi, le nom d'image `alpine` correspond en fait à `docker.io/library/alpine:latest`. La [documentation](https://hub.docker.com/_/alpine/tags) indique les tags existants. En production, pour obtenir des exécutions reproductibles il est conseillé de spécifier la version

Il est possible de choisir la commande à utiliser au lancement du container en l'indiquant après le nom de l'image. Des variables d'environnement peuvent êtrée crées dans le conteneur avec l'option `--env`.

In [13]:
docker container run \
    alpine:3.17.2 uname -a

Unable to find image 'alpine:3.17.2' locally
3.17.2: Pulling from library/alpine
Digest: sha256:69665d02cb32192e52e07644d76bc6f25abeb5410edc1c7a81a10ba3f0efb90a
Status: Downloaded newer image for alpine:3.17.2
Linux 440de1438ffa 5.15.49-linuxkit #1 SMP PREEMPT Tue Sep 13 07:51:32 UTC 2022 aarch64 Linux


In [15]:
docker container run --env FIRSTNAME="John" --env NAME="Doe" \
    alpine:3.17.2 env

PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME=60ed99eeaf92
NAME=Doe
FIRSTNAME=John
HOME=/root


In [17]:
docker container run \
    alpine:3.17.2 sh -c "echo 'Hi !' > hello.txt ; cat hello.txt"

Hi !


Si la commande est interactive (comme un shell bash par exemple il faut préciser `--interactive` ou `-it`) pour utiliser l'entrée et la sortie standard. 

Il est aussi possible de donner un nom unique à conteneur qui peut être utilisé à la place de son ID avec l'option `--name`.

In [21]:
(
cat <<EOF
cd /root
ls -al
EOF
) | docker run --interactive --quiet --name my-bash ubuntu:22.04 bash - 

docker: Error response from daemon: Conflict. The container name "/my-bash" is already in use by container "da7801cca6c30759b086a246af843448331c56bbb23e91312ab883cabe5ad5a5". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


L'exécution d'un conteneur peut être faite tâche de fond avec l'option `--detach` ou `-d`.

Par exemple, pour lancer [redis](https://redis.com/), une base de donnée NoSQL de type clé/valeur :

In [24]:
docker run --detach --name my-redis redis:7.0.9

Unable to find image 'redis:7.0.9' locally
7.0.9: Pulling from library/redis

ba0fb1b5: Pulling fs layer 
abea72ba: Pulling fs layer 
c855d06e: Pulling fs layer 
636102fd: Pulling fs layer 
9543da32: Pulling fs layer 
Digest: sha256:e50c7e23f79ae81351beacb20e004720d4bed657415e68c2b1a2b5557c075ce0
Status: Downloaded newer image for redis:7.0.9
a7064f3a9a309bb2ccff059dd3cf8a47d5084b6f489e22582e54f4633b22ff22


Les conteneurs produisent généralement le log du service sur la sortie standard. Quand `--detach` est utilisé la commande `logs` permet de le consulter (ajouter `-f` pour un suivi en continu).

In [26]:
docker logs my-redis

1:C 13 Mar 2023 07:44:04.119 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
1:C 13 Mar 2023 07:44:04.120 # Redis version=7.0.9, bits=64, commit=00000000, modified=0, pid=1, just started
1:C 13 Mar 2023 07:44:04.120 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
1:M 13 Mar 2023 07:44:04.120 * monotonic clock: POSIX clock_gettime
1:M 13 Mar 2023 07:44:04.120 * Running mode=standalone, port=6379.
1:M 13 Mar 2023 07:44:04.120 # Server initialized
1:M 13 Mar 2023 07:44:04.122 * Ready to accept connections


La sous-commande `ls` permet d'obtenir tous les conteneurs en cours d'exécution.

In [28]:
docker container ls

CONTAINER ID   IMAGE         COMMAND                  CREATED              STATUS              PORTS      NAMES
a7064f3a9a30   redis:7.0.9   "docker-entrypoint.s…"   About a minute ago   Up About a minute   6379/tcp   my-redis


La commande `ls -a` affiche aussi ceux qui se sont arrêtés.

In [30]:
docker container ls -a

CONTAINER ID   IMAGE           COMMAND                  CREATED              STATUS                     PORTS      NAMES
a7064f3a9a30   redis:7.0.9     "docker-entrypoint.s…"   About a minute ago   Up About a minute          6379/tcp   my-redis
da7801cca6c3   ubuntu          "bash -"                 3 minutes ago        Exited (0) 3 minutes ago              my-bash
42708530e6a4   alpine:3.17.2   "sh -c 'echo 'Hi !' …"   4 minutes ago        Exited (0) 4 minutes ago              dazzling_ptolemy
1fee5f5e436e   alpine:3.17.2   "uname -a alpine: sh…"   4 minutes ago        Exited (1) 4 minutes ago              awesome_dewdney
60ed99eeaf92   alpine:3.17.2   "env"                    4 minutes ago        Exited (0) 4 minutes ago              reverent_hypatia
90028ea2041c   alpine:3.17.2   "uname -a alpine: env"   4 minutes ago        Exited (1) 4 minutes ago              stoic_kilby
440de1438ffa   alpine:3.17.2   "uname -a"               4 minutes ago        Exited (0) 4 minutes ago         

Un conteneur s'arrête quand sa commande termine. Il peut être arrêté manuellement avec la sous-commande `stop`.

In [33]:
docker container stop my-redis

my-redis


Un conteneur arrêté peut être relancé avec les même paramètres avec la commande `start`.

In [35]:
docker container start my-redis

my-redis


Un container arrêté peut être détruit avec la commande `rm` en indiquant son Id ou son nom.

In [37]:
docker rm my-bash

my-bash


La commande `exec` permet d'exécuter une commande dans un conteneur en fonctionnement. Par exemple, pour utiliser l'interface en ligne de commande de redis depuis le conteneur `my-redis` (qui exécute déjà le serveur) pour ajouter une valeur de clé "a.b@x.fr".

In [40]:
docker exec my-redis redis-cli SET a.b@x.fr "Pierre,Durand,12"

OK


L'option `--rm` de la commande `run` provoque la suppression du conteneur dès l'arrêt.

On peut alors créer un autre conteneur éphémère pour exécuter la commande de récupération d'une valeur de redis via le réseau (la partie réseau est expliquée plus tard) à partir de l'image du serveur. 

In [42]:
docker run --rm --link my-redis redis:7.0.9 redis-cli -h my-redis GET a.b@x.fr

Pierre,Durand,12


In [43]:
docker container ls 

CONTAINER ID   IMAGE         COMMAND                  CREATED         STATUS          PORTS      NAMES
a7064f3a9a30   redis:7.0.9   "docker-entrypoint.s…"   2 minutes ago   Up 40 seconds   6379/tcp   my-redis


In [44]:
docker container stop my-redis
docker container rm my-redis

my-redis
my-redis


Tous les containeurs arrêtés peuvent être supprimés avec la commande `container prune`.

In [46]:
docker container prune --force

Deleted Containers:
42708530e6a42256a02a43b810f9a75864f6a99d30aab8f05740da07a5bb6e85
1fee5f5e436e46144ca6143240bade8d6b2e28275cbec3761e9347638a1ddf70
60ed99eeaf925c7ddcb628a69583f8de2ecad8af0f3399956ef42858b0e0bbcb
90028ea2041cdd3601547b55c71fe0e774168285dfdd10fdb5fc492f1da5e62c
440de1438ffa05f3084582145ac841d62cd8f201e9f93383c3792b98ae5add7a
4d7a6681d325e73873785701fdf5a2cced0a0c66700f87cc0ff949c9bca67da3
ea44f7979f47dc23be3af67b7d8699a19a4a429473a346d3c597e31b26042b8e

Total reclaimed space: 5B


In [47]:
docker container ls -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


### Gérer les images

Il est possible de lister ou supprimer les images disponibles localement avec la commande `image` et les sous-commandes `ls` et `rm`.

In [49]:
docker image ls

REPOSITORY        TAG           IMAGE ID       CREATED         SIZE
restjpa           latest        d0d3ff53a1a8   2 days ago      281MB
redis             7.0.9         edf4b3932692   11 days ago     111MB
ubuntu            22.04         730eeb702b69   12 days ago     69.2MB
ubuntu            latest        730eeb702b69   12 days ago     69.2MB
curlimages/curl   7.88.1        5880c2bc66d5   2 weeks ago     14.4MB
postgres          15.2-alpine   68d4a8d9d3d9   4 weeks ago     241MB
alpine            3.17.2        d74e625d9115   4 weeks ago     7.46MB
alpine            latest        d74e625d9115   4 weeks ago     7.46MB
hello-world       latest        46331d942d63   11 months ago   9.14kB


In [50]:
docker image rm hello-world redis:7.0.9 alpine

Untagged: hello-world:latest
Untagged: hello-world@sha256:6e8b6f026e0b9c419ea0fd02d3905dd0952ad1feea67543f525c73a0a790fefb
Deleted: sha256:46331d942d6350436f64e614d75725f6de3bb5c63e266e236e04389820a234c4
Deleted: sha256:efb53921da3394806160641b72a2cbd34ca1a9a8345ac670a85a04ad3d0e3507
Untagged: redis:7.0.9
Untagged: redis@sha256:e50c7e23f79ae81351beacb20e004720d4bed657415e68c2b1a2b5557c075ce0
Deleted: sha256:edf4b3932692a4fdbd04e0bdb070e95f9e6ad4b534739a74d9da1b92215314c6
Deleted: sha256:91bff5f3e5746daba37a232fb1eb74836b6475ec31bfcb35ac6e0ea4d0bfa3c4
Deleted: sha256:2666794cc78a523f303e45aad123e15c751adaff07b958e09c5a19d06b88f5f4
Deleted: sha256:1bab36e27e8fec54fe04a4087e4a653b285b5a2ffdf3fc21c51360a9917bd416
Deleted: sha256:fabe0a0282bcf61eea2d81ad3a734d7f27d93c22f9893bd70ef68f0c6d98c98c
Deleted: sha256:098e5ea0c61165e2bd555c1a9ffaccc2c8cc59d032c253d82903508dbf477b3f
Deleted: sha256:ace73ce182d142e2b77b7b617660b45f8a766cc9c5d36174f4a8b686e3af695e
Untagged: alpine:latest


In [51]:
docker image prune --force

Total reclaimed space: 0B


### Volumes

Un conteneur est généralement éphémère et sans état pour pouvoir être arrêté et relancé sans perte de données. Les fichiers créés devant être persistants doivent donc l'être en dehors du conteneur. Pour cela, on utilise les montage de systeme d fichiers (bind mount) ou les volumes. 

Un bind volume, c'est-à-dire un montage du système de fichier de l'hôte vers un répertoire du conteneur, est définit avec l'option `-v <source>:<destination>` de la commande `run` que l'on peut uiliser plusieurs fois. Le chemin doit être absolu mais peut utiliser la variable d'environement `${PWD}` pour simuler un chemin relatif. 

Dans l'exemple suivant, un premier conteneur crée le fichier `text.txt` dans un volume monté de l'hôte (le répertoire `/tmp/mydata`) dans le conteneur (à l'emplacement `/data`). Le conteneur est ensuite détruit.
Un second conteneur monte le même répertoire de l'hôte (`/tmp/mydata` dans le nouveau conteneur mais dans `/databis`) et affiche le contenu.

In [7]:
docker container run --rm \
    -v /tmp/mydata:/data \
    ubuntu:22.04 \
        bash -c "echo 'hello'>/data/test.txt"

In [8]:
docker container run --rm \
    -v /tmp/mydata:/databis \
    ubuntu:22.04 \
        cat /databis/test.txt

hello


un volume peut aussi être un volume nommé géré par le docker daemon de façon transparente pour l'utilisateur. Il n'est alors plus aussi simple de partager des fichiers entre l'hôte et les conteneurs mais il n'y a plus de problème d'UID du propriétaire ni de droits. Il s'agit de la meilleure solution quand le docker daemon est exécuté sur un cluster et non sur une seule machine car alors le système de fichier peut être distribué.

L'exemple suivant exécute une base de données via l'image de [PostgreSQL](https://hub.docker.com/_/postgres) dans un premier conteneur. Le volume nommé `postgres-test-data` est monté dans le répertoire indiqué dans la documentation de l'image pour contenir la base de données (`/var/lib/postgresql/data`). Comme celui-ci est initialement vide, l'image est construite pour créer alors automatiquement une base de données à partir des variables d'environnement fournies. 

In [43]:
docker run -d --rm \
  --name postgres-test \
  --env POSTGRES_PASSWORD=mysecretpassword \
  -v postgres-test-data:/var/lib/postgresql/data \
  postgres:15.2

d3b2f0eb2fb46a122756ea7a43f95971b730153b821cfbd23854ce7622079272


Il est possible de monter le même volume et un bind volume dans une autre conteneur par exemple pour faire une sauvegarde. En pratique, on préfère `pg_dump`, mais nous l'illustrons ici avec un simple tar. Dans le cas d'une base de données relationnelles l'arrêt du conteneur (ou un snapshot) est obligatoire pour garder la cohérence. 

In [45]:
docker container stop postgres-test 

docker run --rm \
    -v postgres-test-data:/var/lib/postgresql/data \
    -v /tmp/backup:/backup \
    ubuntu:22.04 \
    tar zcf --totals /backup/mydb.tar.gz /var/lib/postgresql/data

postgres-test
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets


In [46]:
docker stop postgres-test

Error response from daemon: No such container: postgres-test


Ensuite, un autre conteneur PostgreSQL peut être lancé en utilisant la même base.

In [48]:
docker run -d --rm \
  --name postgres-test \
  --env POSTGRES_PASSWORD=mysecretpassword \
  -v postgres-test-data:/var/lib/postgresql/data \
  postgres:15.2

17b87f14e8f2371ada28cf5f30ebe7974aef2d8c0a99f814ac9e9b5b7cf74194


In [49]:
docker logs postgres-test


PostgreSQL Database directory appears to contain a database; Skipping initialization

2023-03-13 17:18:21.256 UTC [1] LOG:  starting PostgreSQL 15.2 (Debian 15.2-1.pgdg110+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit
2023-03-13 17:18:21.257 UTC [1] LOG:  listening on IPv4 address "0.0.0.0", port 5432
2023-03-13 17:18:21.257 UTC [1] LOG:  listening on IPv6 address "::", port 5432
2023-03-13 17:18:21.258 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
2023-03-13 17:18:21.264 UTC [29] LOG:  database system was shut down at 2023-03-13 17:17:55 UTC
2023-03-13 17:18:21.269 UTC [1] LOG:  database system is ready to accept connections


In [50]:
docker stop postgres-test

postgres-test


Il est possible de lister et de supprimer les volumes avec les commandes `ls` et `rm`. Les volumes dont les noms sont des hachés appelés volumes anonymes sont présentés plus tard. 

In [52]:
docker volume ls

DRIVER    VOLUME NAME
local     ad39346c95258ba7e3e9452050a5e30e6c81bfcafdc4fb865ae8795f644d67b5
local     postgres-01-data
local     postgres-test-data


In [53]:
docker volume rm postgres-test-data

postgres-test-data


In [54]:
docker volume ls

DRIVER    VOLUME NAME
local     ad39346c95258ba7e3e9452050a5e30e6c81bfcafdc4fb865ae8795f644d67b5
local     postgres-01-data


### Réseau

Chaque conteneur dispose d'une carte réseau virtuelle et d'au moins une adresse IP. Un conteneur peut utiliser le réseau de l'hôte (`--host`) mais généralement il appartient à un ou plusieurs réseau virtuels. Si aucun n'est précisé expliciement, le conteneur appartient à un réseau par défaut. Comme les conteneurs sont éphémères et que les adresses IP changent, elles ne sont généralement pas utilisées directement. 

Les réseaux peuvent être créés, listés et détruits avec les commandes `network {create, ls, rm}`. 

Ainsi deux conteneurs qui exécutent un shell sont sur le réseau par défaut.

In [67]:
 docker run -dit --rm --name alpine1 alpine ash
 docker run -dit --rm --name alpine2 alpine ash

a5c823e506c9d895c4556d9def40dbc065f26a8c394cfb25023fde4f71dd7a59
28f17fdef7b951599dbefa26b6840aede5858f653e4744b9368cb507ed270211


In [68]:
docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
3a3f9fe39100   backnet   bridge    local
25a6bacef95b   bridge    bridge    local
290659c62698   host      host      local
e91b5de18bd5   none      null      local


La commande `docker inspect {container, volume, network} <name|ID>` permet de consulter le détails d'un objet docker. `jq` est un outils pour traiter des documents json.

In [70]:
docker network inspect bridge|jq

[
  {
    "Name": "bridge",
    "Id": "25a6bacef95b3886fdaf288237c40292d019a2f681449e6f82f0659c8a616209",
    "Created": "2023-03-10T15:50:32.692030545Z",
    "Scope": "local",
    "Driver": "bridge",
    "EnableIPv6": false,
    "IPAM": {
      "Driver": "default",
      "Options": null,
      "Config": [
        {
          "Subnet": "172.17.0.0/16"
        }
      ]
    },
    "Internal": false,
    "Attachable": false,
    "Ingress": false,
    "ConfigFrom": {
      "Network": ""
    },
    "ConfigOnly": false,
    "Containers": {
      "28f17fdef7b951599dbefa26b6840aede5858f653e4744b9368cb507ed270211": {
        "Name": "alpine2",
        "EndpointID": "fc5b045b28a63f50301cd44115f0c00c88fc862e81b9ff94edf8d21701c3f546",
        "MacAddress": "02:42:ac:11:00:03",
        "IPv4Address": "172.17.0.3/16",
        "IPv6Address": ""
      },
      "a5c823e506c9d895c4556d9def40dbc065f26a8c394cfb25023fde4f71dd7a59": {
        "Name": "alpine1",
        "EndpointID": "703c99f9b8a46e3edce39d

Avec l'option `--link` lors du `run`un conteneur peut utiliser le nom d'un autre conteneur comme nom d'hôte. Ici on utilise un image qui contient des utilitaires réseaux.

In [78]:
docker run --rm \
    --link alpine1 --link alpine2 \
    nicolaka/netshoot:latest \
        ping -c 1 alpine1 

PING alpine1 (172.17.0.2) 56(84) bytes of data.
64 bytes from alpine1 (172.17.0.2): icmp_seq=1 ttl=64 time=0.423 ms

--- alpine1 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.423/0.423/0.423/0.000 ms


In [79]:
docker container stop alpine1 alpine2

alpine1
alpine2


#### entre conteneurs

Pour que deux conteneurs puissent communiquer simplement, le mieux est qu'ils appartiennent à un même réseau et qu'ils soient nommés. Le nom du conteneur peut alors être utilisé comme un nom d'hôte sur le réseau.

si aucun réseau n'est précisé, le conteneur se trouvent dans le réseau par défaut. Il est alors obligatoire d'utiliser l'option `--link` qui est dépréciée. 

In [82]:
docker network create mynet

15d509717a9ef3377d4bb1c363b2b7a04b73613bbfd171b1949351387bb4fb93


On peut alors exécuter un conteneur nommé `nginx-01` (il fournit un serveur web nginx) sur ce réseau avec l'option `--network`.

In [84]:
docker run --rm -d --name nginx-01 \
           --network mynet \
           nginx:1.23

83741e9033df8a07f68f416c59c349b5162fa95e343a4521476e1541de9bbfce


In [85]:
docker network inspect mynet|jq

[
  {
    "Name": "mynet",
    "Id": "15d509717a9ef3377d4bb1c363b2b7a04b73613bbfd171b1949351387bb4fb93",
    "Created": "2023-03-13T17:30:10.2379153Z",
    "Scope": "local",
    "Driver": "bridge",
    "EnableIPv6": false,
    "IPAM": {
      "Driver": "default",
      "Options": {},
      "Config": [
        {
          "Subnet": "192.168.0.0/20",
          "Gateway": "192.168.0.1"
        }
      ]
    },
    "Internal": false,
    "Attachable": false,
    "Ingress": false,
    "ConfigFrom": {
      "Network": ""
    },
    "ConfigOnly": false,
    "Containers": {
      "83741e9033df8a07f68f416c59c349b5162fa95e343a4521476e1541de9bbfce": {
        "Name": "nginx-01",
        "EndpointID": "1fda3db8766adbc5fc7c74b8f834abebd9bd876d1fdda61c5102dc7089054c0d",
        "MacAddress": "02:42:c0:a8:00:02",
        "IPv4Address": "192.168.0.2/20",
        "IPv6Address": ""
      }
    },
    "Options": {},
    "Labels": {}
  }
]


In [94]:
docker run --quiet --detach --rm --name my-tshark \
           --network container:nginx-01 \
           nicolaka/netshoot:v0.9 \
             tshark -w /tmp/capture-output.pcap

91f87edaca08d83014b2618a0988ac270fcec7f83118cdd50b1bda91df25f85e


ce serveur peut être atteint depuis un autre conteneur sur le même réseau en utilisant son nom. Ici on exécute la commande `pandoc` qui convertit un site web en texte :

In [96]:
 docker run --network mynet --rm pandoc/core:3.1 --to plain http://nginx-01

Welcome to nginx!

If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.

For online documentation and support please refer to nginx.org.
Commercial support is available at nginx.com.

Thank you for using nginx.


In [97]:
docker exec -it my-tshark tshark -r /tmp/capture-output.pcap
docker stop my-tshark

    1 0.000000000 02:42:c0:a8:00:03 → Broadcast    ARP 42 Who has 192.168.0.2? Tell 192.168.0.3
    2 0.000017791 02:42:c0:a8:00:02 → 02:42:c0:a8:00:03 ARP 42 192.168.0.2 is at 02:42:c0:a8:00:02
    3 0.000039750  192.168.0.3 → 192.168.0.2  TCP 74 47330 → 80 [SYN] Seq=0 Win=64240 Len=0 MSS=1460 SACK_PERM TSval=1857303026 TSecr=0 WS=128
    4 0.000055958  192.168.0.2 → 192.168.0.3  TCP 74 80 → 47330 [SYN, ACK] Seq=0 Ack=1 Win=65160 Len=0 MSS=1460 SACK_PERM TSval=2744687250 TSecr=1857303026 WS=128
    5 0.000073958  192.168.0.3 → 192.168.0.2  TCP 66 47330 → 80 [ACK] Seq=1 Ack=1 Win=64256 Len=0 TSval=1857303026 TSecr=2744687250
    6 0.004066916  192.168.0.3 → 192.168.0.2  HTTP 123 GET / HTTP/1.1 
    7 0.004084041  192.168.0.2 → 192.168.0.3  TCP 66 80 → 47330 [ACK] Seq=1 Ack=58 Win=65152 Len=0 TSval=2744687254 TSecr=1857303030
    8 0.004339583  192.168.0.2 → 192.168.0.3  TCP 304 HTTP/1.1 200 OK  [TCP segment of a reassembled PDU]
    9 0.004356166  192.168.0.3 → 192.168.0.2  TCP 66 4733

In [98]:
docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
3a3f9fe39100   backnet   bridge    local
25a6bacef95b   bridge    bridge    local
290659c62698   host      host      local
15d509717a9e   mynet     bridge    local
e91b5de18bd5   none      null      local


In [100]:
docker stop nginx-01
docker network rm mynet

Error response from daemon: No such container: nginx-01
Error response from daemon: network mynet not found


#### entre l'hôte et les conteneurs

Il est possible de mettre en place une redirection de ports entre l'hôte et les conteneurs avec l'option `-p <port hote>:<port conteneur>` à la création. Cela permet d'accéder aux services docker depuis l'hôte ou le réseau local. 

On peut donc lancer deux serveur nginx de deux versions différents qui écoutent chacun sur le port 80 de leur conteneur respectif mais qui sont accessibles depuis les port 8081 et 8082 de l'hôte.

In [121]:
docker run --rm -d --name nginx-01 -p 8081:80 nginx:1.22
docker run --rm -d --name nginx-02 -p 8082:80 nginx:1.23

879095a27359a005393beedac39652b0d3a22f3a87abf4be110bc69e060517b6
a441e69c2e25e72f8f466528fcead02f1d9b993de8fea7490ad93a8cd6072294


On utilise ici une technique pour accéder à l'hôte depuis les conteneurs.

In [124]:
docker run --rm \
    --add-host=host.docker.internal:host-gateway \
    curlimages/curl:7.88.1 --silent --head host.docker.internal:8081|head -n 2
    
docker run --rm \
    --add-host=host.docker.internal:host-gateway \
    curlimages/curl:7.88.1 --silent --head host.docker.internal:8082|head -n 2

HTTP/1.1 200 OK
Server: nginx/1.22.1
HTTP/1.1 200 OK
Server: nginx/1.23.3


In [125]:
docker stop nginx-01 nginx-02

nginx-01
nginx-02


### Illustration avec une base de données relationnelles

Pour illustrer les concepts précédents, nous allons créer une base de données Postgresql à partir de l'[image officielle](https://hub.docker.com/_/postgres) dans un sous-réseau `backnet` et dont les données seront persistées dans un volume nommé `postgres-01-data`.

In [126]:
docker network create backnet

Error response from daemon: network with name backnet already exists


In [127]:
docker run -d --rm \
  --name postgres-01 \
  --network backnet \
  --env POSTGRES_USER=dba \
  --env POSTGRES_PASSWORD=mysecretpassword \
  --env POSTGRES_DB=mydb \
  -v postgres-01-data:/var/lib/postgresql/data \
  postgres:15.2

docker: Error response from daemon: Conflict. The container name "/postgres-01" is already in use by container "899dee12389ab75dd6283a2d7cc1e10a130877bc9d822f4513ce82c9a4a4fef8". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


Il est ensuite possible d'interroger cette base de données avec la commande `psql` exécutée dans un autre conteneur ephémère. 

In [129]:
sleep 2 

docker run --rm \
    --network backnet \
    --env PGPASSWORD=mysecretpassword \
    postgres:15.2 \
        psql -h postgres-01 -U dba mydb -c \
            "CREATE TABLE IF NOT EXISTS 
                    PERSON(ID serial PRIMARY KEY,NAME VARCHAR NOT NULL);
            INSERT INTO PERSON (NAME) VALUES ('Pierre');
            INSERT INTO PERSON (NAME) VALUES ('Marie');"

NOTICE:  relation "person" already exists, skipping
CREATE TABLE
INSERT 0 1
INSERT 0 1


Le conteneur de la base de donnée peut être détruit (cf --rm) et un autre créé avec le même volume donc sans perte de données. 

In [29]:
docker stop postgres-01

docker run -d --rm \
  --name postgres-01 \
  --network backnet \
  --env POSTGRES_USER=dba \
  --env POSTGRES_PASSWORD=mysecretpassword \
  --env POSTGRES_DB=mydb \
  -v postgres-01-data:/var/lib/postgresql/data \
  postgres:15.2

postgres-01
899dee12389ab75dd6283a2d7cc1e10a130877bc9d822f4513ce82c9a4a4fef8


In [30]:
sleep 2 

docker run --rm \
    --network backnet \
    --env PGPASSWORD=mysecretpassword \
    postgres:15.2 \
        psql -h postgres-01 -U dba mydb -c \
            "SELECT * FROM PERSON;"

 id |  name  
----+--------
  1 | Pierre
  2 | Marie
(2 rows)



On peut aussi utiliser un bind volume pour faire une sauvegarde sur l'hôte.

In [132]:
docker run --rm \
    -v /tmp/backup:/backup \
    --network backnet \
    --env PGPASSWORD=mysecretpassword \
    postgres:15.2 \
        sh -c "pg_dumpall -h postgres-01 -U dba|gzip > \
               /backup/pg01-$(date +'%Y%m%d_%H%M%S').backup.gz"

Pour finir, le conteneur postgres-01 est arrêté et donc détruit, ainsi que le réseau.\
Dans notre exemple, le volume postgres-01-data est aussi détruit mais ***attention les données perdues***. 

In [134]:
docker stop postgres-01
docker network rm backnet
docker volume rm postgres-01-data

postgres-01
backnet
postgres-01-data
